In [1]:
#librerias necesarias
import pandas as pd
import os
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Ruta base
base_path = "/content/drive/MyDrive/movie_project"

# Cargar MovieLens
movies = pd.read_csv(f"{base_path}/movies.csv")
links = pd.read_csv(f"{base_path}/links.csv")
ratings = pd.read_csv(f"{base_path}/ratings.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")
genome_scores = pd.read_csv(f"{base_path}/genome-scores.csv")
genome_tags = pd.read_csv(f"{base_path}/genome-tags.csv")

# Cargar Movie Genre from Poster
movie_genre = pd.read_csv(f"{base_path}/MovieGenre.csv", encoding='latin1')

# Cargar nombres de imágenes disponibles (formato jpg, nombradas por tmdbId)
poster_folder = f"{base_path}/SampleMoviePosters"
image_files = os.listdir(poster_folder)
poster_tmdb_ids = [int(f.split('.')[0]) for f in image_files if f.endswith('.jpg')]

# ======================== MOVIES ========================
print("\n===== DATOS DE PELÍCULAS (movies.csv) =====")
print("Tamaño:", movies.shape)
print("Columnas:", list(movies.columns))
print("\nPrimeras filas:")
print(movies.head())

# ======================== LINKS ========================
print("\n===== DATOS DE ENLACES (links.csv) =====")
print("Tamaño:", links.shape)
print("Columnas:", list(links.columns))
print("\nPrimeras filas:")
print(links.head())

# ======================== RATINGS ========================
print("\n===== CALIFICACIONES (ratings.csv) =====")
print("Tamaño:", ratings.shape)
print("Columnas:", list(ratings.columns))
print("\nPrimeras filas:")
print(ratings.head())

# ======================== TAGS ========================
print("\n===== ETIQUETAS DE USUARIO (tags.csv) =====")
print("Tamaño:", tags.shape)
print("Columnas:", list(tags.columns))
print("\nPrimeras filas:")
print(tags.head())

# ======================== GENOME TAGS ========================
print("\n===== GENOME TAGS (genome-tags.csv) =====")
print("Tamaño:", genome_tags.shape)
print("Columnas:", list(genome_tags.columns))
print("\nPrimeras filas:")
print(genome_tags.head())

# ======================== GENOME SCORES ========================
print("\n===== GENOME SCORES (genome-scores.csv) =====")
print("Tamaño:", genome_scores.shape)
print("Columnas:", list(genome_scores.columns))
print("\nPrimeras filas:")
print(genome_scores.head())

# ======================== POSTER GENRES ========================
print("\n===== GÉNEROS DESDE PÓSTERS (MovieGenre.csv) =====")
print("Tamaño:", movie_genre.shape)
print("Columnas:", list(movie_genre.columns))
print("\nPrimeras filas:")
print(movie_genre.head())

# ======================== PÓSTERS DISPONIBLES ========================
print("\n===== PÓSTERS EN LA CARPETA =====")
print(f"Cantidad de imágenes: {len(poster_tmdb_ids)}")
print("Primeros 10 tmdbId de imágenes disponibles:")
print(poster_tmdb_ids[:10])



===== DATOS DE PELÍCULAS (movies.csv) =====
Tamaño: (62423, 3)
Columnas: ['movieId', 'title', 'genres']

Primeras filas:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

===== DATOS DE ENLACES (links.csv) =====
Tamaño: (62423, 3)
Columnas: ['movieId', 'imdbId', 'tmdbId']

Primeras filas:
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  118

In [9]:
# Asegurar que tmdbId sea entero en ambos datasets
links['tmdbId'] = pd.to_numeric(links['tmdbId'], errors='coerce').astype('Int64')
movie_genre['imdbId'] = pd.to_numeric(movie_genre['imdbId'], errors='coerce')

# Merge entre movie_genre y links para traer tmdbId y movieId
df_merged = pd.merge(movie_genre, links, on='imdbId', how='inner')

# Traer título y géneros de MovieLens
df_merged = pd.merge(df_merged, movies, on='movieId', how='left')  # trae 'title' y 'genres'

# Filtrar solo los que tienen tmdbId en las imágenes disponibles
df_merged = df_merged[df_merged['tmdbId'].isin(poster_tmdb_ids)].copy()

# Mostrar info
print("\n===== DATOS UNIFICADOS CON IMAGEN =====")
print("Tamaño:", df_merged.shape)
print("Columnas:", list(df_merged.columns))
print(df_merged[['tmdbId', 'title', 'IMDB Score', 'Genre', 'genres']].head())

#guardamos la data final que vamos a utilizar
df_merged.to_csv("/content/drive/MyDrive/movie_project/movies_posters.csv", index=False)





===== DATOS UNIFICADOS CON IMAGEN =====
Tamaño: (19715, 10)
Columnas: ['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster', 'movieId', 'tmdbId', 'title', 'genres']
   tmdbId                               title  IMDB Score  \
0     862                    Toy Story (1995)         8.3   
1    8844                      Jumanji (1995)         6.9   
2   15602             Grumpier Old Men (1995)         6.6   
3   31357            Waiting to Exhale (1995)         5.7   
4   11862  Father of the Bride Part II (1995)         5.9   

                        Genre                                       genres  
0  Animation|Adventure|Comedy  Adventure|Animation|Children|Comedy|Fantasy  
1     Action|Adventure|Family                   Adventure|Children|Fantasy  
2              Comedy|Romance                               Comedy|Romance  
3        Comedy|Drama|Romance                         Comedy|Drama|Romance  
4       Comedy|Family|Romance                                       Com

In [14]:
df = pd.read_csv("/content/drive/MyDrive/movie_project/movies_posters.csv")
print("Tamaño del dataset:", df.shape)
df.head()

Tamaño del dataset: (19715, 9)


,imdbId,Imdb Link,Title,IMDB Score,Poster,movieId,tmdbId,title,genres
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,https://images-na.ssl-images-amazon.com/images...,1,862,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,https://images-na.ssl-images-amazon.com/images...,2,8844,Jumanji (1995),Adventure|Children|Fantasy
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,https://images-na.ssl-images-amazon.com/images...,3,15602,Grumpier Old Men (1995),Comedy|Romance
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,https://images-na.ssl-images-amazon.com/images...,4,31357,Waiting to Exhale (1995),Comedy|Drama|Romance
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,https://images-na.ssl-images-amazon.com/images...,5,11862,Father of the Bride Part II (1995),Comedy


In [17]:
print(df_merged.columns.tolist())
print(df_merged.columns)
df_merged.to_csv("/content/drive/MyDrive/movie_project/movies_posters.csv", index=False)



['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Poster', 'movieId', 'tmdbId', 'title', 'genres']
Index(['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Poster', 'movieId',
       'tmdbId', 'title', 'genres'],
      dtype='object')
